In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   30G  311G   9% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       365G   32G  334G   9% /opt/bin
shm              10G     0   10G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G   14G  1.5G  91% /content/gdrive


In [0]:
cat ../../../etc/fstab

# UNCONFIGURED FSTAB FOR BASE SYSTEM
tmpfs /dev/shm tmpfs defaults,size=10g 0 0
tmpfs /dev/shm tmpfs defaults,size=16g 0 0


In [0]:
!echo "tmpfs /dev/shm tmpfs defaults,size=16g 0 0" >> ../../../etc/fstab

In [0]:
cat ../../../etc/fstab

# UNCONFIGURED FSTAB FOR BASE SYSTEM
tmpfs /dev/shm tmpfs defaults,size=10g 0 0
tmpfs /dev/shm tmpfs defaults,size=16g 0 0
tmpfs /dev/shm tmpfs defaults,size=16g 0 0


In [0]:
!mount -o remount /dev/shm

In [0]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   30G  311G   9% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       365G   32G  334G   9% /opt/bin
shm              10G     0   10G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G   14G  1.5G  91% /content/gdrive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import copy


In [0]:
def L_model_forward(X, parameters):

    x0 = X
    caches = {}
#    caches = []
    # Gotta save the inputs as the first activations
    caches['A0'] = x0
#    print('Input size..'+str(x0.shape))

    for i in range(1, int(len(parameters.keys())/2)+1):
#        print('Max value in W'+str(i)+'....'+str(np.max(parameters['W'+str(i)].flatten())))
        zl = np.matmul(parameters['W'+str(i)], x0) + parameters['b'+str(i)]
        if (i == int(len(parameters.keys())/2)):
            # Output layer Activation is sigmoid        
            x0 = sigmoid(zl)
#            print('Fwd prop Layer..'+str(i)+'...Sigmoid..'+str(parameters['W'+str(i)].shape))            
        else:            
#            print('Fwd prop Layer..'+str(i)+'...Relu..'+str(parameters['W'+str(i)].shape))            
            x0 = relu(zl)

        caches['A'+str(i)]=x0
        caches['Z'+str(i)]=zl
        caches['W'+str(i)]=parameters['W'+str(i)]
        caches['b'+str(i)]=parameters['b'+str(i)]        

    x0[x0 <= 0.001] = 0.001
    x0[x0 >= 0.999] = 0.999

    AL = x0     
    
#    print('Caches..', len(caches))
    return AL, caches 


In [0]:
def initialize_parameters_deep(layers_dims):

    num_layers = len(layers_dims)
#    print(num_layers)
    parameters = {}
    
    for i in range(num_layers-1):
        parameters['W'+str(i+1)] = np.random.randn(layers_dims[i+1], layers_dims[i]) * np.sqrt(2/(layers_dims[i+1]+layers_dims[i]))
        parameters['b'+str(i+1)] = np.zeros((layers_dims[i+1], 1))
#        print(parameters['b'+str(i+1)].shape)
        
    return parameters 



In [0]:
def update_parameters(parameters, grads, learning_rate):

    for i in range(1, int(len(parameters.keys())/2)+1):
#        print('Updating Layer..'+str(i))
        parameters['W'+str(i)] = parameters['W'+str(i)] - (learning_rate * grads['dW'+str(i)])
        parameters['b'+str(i)] = parameters['b'+str(i)] - (learning_rate * grads['db'+str(i)])

    return parameters



In [0]:
# Backward propagation (Sigmoid -> Relu -> Relu..)
def L_model_backward(AL, Y, caches):

    grads = {}
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))
    numlayers = int(np.floor(len(caches)/4))   

    da_next = np.divide(1-Y, 1-AL) - (np.divide(Y, AL))

    for i in range(numlayers, 0, -1):
        # Last layer has a different activation
        if (i == numlayers):
            gprime_curr = sigmoidDeriv(caches['Z'+str(i)])
        else:
            gprime_curr = reluDerivative(caches['Z'+str(i)])

        dz = np.multiply(da_next, gprime_curr)

        # Activations coming into the first layer are the inputs
        temp_al_prev = np.transpose(caches['A'+str(i-1)])

        dW = np.divide(np.matmul(dz, temp_al_prev), m)
        db = np.divide(np.sum(dz, axis=1, keepdims=True), m)
        
        da_prev = np.matmul(np.transpose(caches['W'+str(i)]), dz)
        
        da_next = da_prev
        grads['dW'+str(i)] = dW
        grads['db'+str(i)] = db

    return grads


In [0]:
# Backward propagation (Sigmoid -> Relu -> Relu..)
def L_model_backward_L2_reg(AL, Y, caches, L2_lambd=200):

    grads = {}
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))
    numlayers = int(np.floor(len(caches)/4))   

    da_next = np.divide(1-Y, 1-AL) - (np.divide(Y, AL))

    for i in range(numlayers, 0, -1):
        # Last layer has a different activation
        if (i == numlayers):
            gprime_curr = sigmoidDeriv(caches['Z'+str(i)])
        else:
            gprime_curr = reluDerivative(caches['Z'+str(i)])

        dz = np.multiply(da_next, gprime_curr)

        # Activations coming into the first layer are the inputs
        temp_al_prev = np.transpose(caches['A'+str(i-1)])

        dW = np.divide(np.matmul(dz, temp_al_prev), m)
        # Gradient for W here is L2 regularized
        L2term = (L2_lambd * caches['W'+str(i)])/m
        dW += L2term
        
        db = np.divide(np.sum(dz, axis=1, keepdims=True), m)
        
        da_prev = np.matmul(np.transpose(caches['W'+str(i)]), dz)
        
        da_next = da_prev
        grads['dW'+str(i)] = dW
        grads['db'+str(i)] = db

    return grads


In [0]:

def sigmoid(z):
    sig = 1/(1 + np.exp(-z))
    return sig

def relu(a):
    rel = np.maximum(a, 0)
    return rel

def reluDerivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def sigmoidDeriv(z):
    sd = sigmoid(z)*(1 - sigmoid(z))
    return sd



In [0]:
# Logistic regression cost for now
def compute_cost(AL, Y):
    
#    cost = np.sum(np.multiply((1-Y),np.log((1-AL))) + np.multiply((Y),np.log((AL))))
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))
    
    cost = -1/m * (np.sum(np.multiply(1-Y, np.log(1-AL))) + np.sum(np.multiply(Y, np.log(AL))))
    
    # Also return success rate
    x = (AL >= 0.5)

    success = 1 - (np.count_nonzero(Y - x)/m)
    
    return cost, success


In [0]:
# Logistic regression cost for now
def compute_L2_reg_cost(AL, Y, parameters, L2_lambd = 200):
    
#    cost = np.sum(np.multiply((1-Y),np.log((1-AL))) + np.multiply((Y),np.log((AL))))
    m = Y.size
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))

    # Cross entropy cost    
    cost = -1/m * (np.sum(np.multiply(1-Y, np.log(1-AL))) + np.sum(np.multiply(Y, np.log(AL))))
    
    # Compute the L2 reg cost component
    L2cost = 0
    for i in range(1, int(len(parameters.keys())/2)+1):
      L2cost += np.sum(np.square(parameters['W'+str(i)]))

    L2cost = (L2_lambd * L2cost)/(2*m)
    
    # Also return success rate
    x = (AL >= 0.5)

    success = 1 - (np.count_nonzero(Y - x)/m)
    
    return (cost + L2cost), success


In [0]:
## Global variables are defined in the class definition
class Deep_NN_frmwrk:
    
    def __init__(self, name):
        self.name = name
        # self.localfolder = '/Users/administrator/Documents/Python/dim_128_mono';
        self.localfolder = 'gdrive/My Drive/';
        self.train_data_file = 'X_K_mono_train_P3.dat'
        self.train_labels_file = 'Y_K_mono_train_P3.dat'
        self.test_data_file = 'X_K_mono_test.dat'
        self.test_labels_file = 'Y_K_mono_test.dat'

        
        # Define input feature length - number of pixels in the images here
        self.img_h = 128
        self.img_w = 128
        self.img_d = 1
    
        dim_1 = self.img_h * self.img_w * self.img_d
        self.layers_dims = [dim_1, 25, 13, 5, 1] #  5-layer model
    
        ### Train data
        self.numtrainfiles = 3
        self.kfolds = 20
        self.numbatches = 15 
        self.learning_rate = 0.005
        self.epochs = 100 # per data set
        self.print_cost = 1
        self.use_saved_params = 0
        self.L2_lambd = 0.8
        
        # To hold runtime stuff
        self.costs = []                         # keep track of cost
        self.train_successes = []
        self.xval_successes = []

        # keep track of status
        self.starttime = time.time()
        self.curr_epoch = 0

    def loadSavedParams(self):
    
        num_layers = len(self.layers_dims)
        
        ### Load parameters from training +'.dat'
        parameters = {}
        for i in range(num_layers-1):
            parameters['W'+str(i+1)] = np.loadtxt(self.localfolder+'Out_W'+str(i+1)+'.dat')
            if (len(parameters['W'+str(i+1)].shape) < 2):
                parameters['W'+str(i+1)] = np.reshape(parameters['W'+str(i+1)], (1, len(parameters['W'+str(i+1)])))
                
            parameters['b'+str(i+1)] = np.loadtxt(self.localfolder+'Out_b'+str(i+1)+'.dat')
            if (len(parameters['b'+str(i+1)].shape) < 1):
                parameters['b'+str(i+1)] = np.reshape(parameters['b'+str(i+1)], (1, 1))
            else:
                parameters['b'+str(i+1)] = np.reshape(parameters['b'+str(i+1)], (len(parameters['b'+str(i+1)]), 1))            
        
        return parameters
    
    def saveParams(self, params):
        # Save parameters from training
        for i in range(1,int(len(params.keys())/2)+1):
            np.savetxt(self.localfolder+'Out_W'+str(i)+'.dat', params['W'+str(i)])
            np.savetxt(self.localfolder+'Out_b'+str(i)+'.dat', params['b'+str(i)])
                
            
    def showImageData(X, Y, t_size):
        
    #    # To view the images, make sure labeling is correct
        imgi = int(input('Enter an index..'))
        while (imgi != 99):
            print('Index chosen is...', imgi)
            print('Label is...', Y[imgi])
            reimg = Image.fromarray(np.reshape(X[:,imgi],(t_size)))
    #        t_size = (320,320)
    #        reimg = reimg.resize(t_size, Image.ANTIALIAS)
            plt.imshow(reimg)
            imgi = int(input('Enter an index..'))
    
    def splitKfold(self, X, Y, j, m):
    
        # K-fold xval, separate into train and x_val
        # If no kfold just use 10% as xval
        if (self.kfolds == 1):
            xval = int(m/10)
            x_train = X[:,0:m-xval]
            y_train = Y[0:m-xval]    
            x_xval = X[:,m-xval+1:]
            y_xval = Y[m-xval+1:]
        else:
            xval = int(m/self.kfolds)
            tmp_range = range(j*xval, (j+1)*xval)
            x_xval = X[:, tmp_range]
            y_xval = Y[tmp_range]
            # splits X and Y
            if (j==0):
                tmp_range = range((j+1)*xval+1, m)
                x_train = X[:, tmp_range]
                y_train = Y[tmp_range]
            else:
                tmp_range1 = range(0, j*xval)
                tmp_range2 = range((j+1)*xval+1, m)
                x_train = np.concatenate((X[:, tmp_range1], X[:, tmp_range2]), axis=1)
                y_train = np.concatenate((Y[tmp_range1], Y[tmp_range2]), axis=0)
                
    #    print(x_xval.shape, y_xval.shape, x_train.shape, y_train.shape)
        return x_xval, y_xval, x_train, y_train
    
    def L_layer_model_Kfold(self, X, Y, parameters):#lr was 0.009
    #    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
        
        # Parameters initialization
        if (len(parameters) == 0):
            parameters = initialize_parameters_deep(self.layers_dims)
            
        newparams = copy.deepcopy(parameters)

        m = X.shape[1]
            
        # Loop (gradient descent)
        for i in range(0, self.kfolds): 
    
            X_xval, Y_xval, X_train, Y_train = self.splitKfold(X, Y, i, m)

            # Batch training, numbatches is 1 if no batch separation
            batch_size = int(X_train.shape[1]/self.numbatches)

            for k in range(0, self.numbatches):

                X_batch = X_train[:,(k*batch_size):((k+1)*batch_size-1)]
                Y_batch = Y_train[(k*batch_size):((k+1)*batch_size-1)]

                # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.            
                AL, caches = L_model_forward(X_batch, newparams)

                # Compute cost
#                cost, train_success = compute_cost(AL, Y_batch)
                # Compute cost with L2 reg
                cost, train_success = compute_L2_reg_cost(AL, Y_batch, newparams, self.L2_lambd)

                # Backward propagation
#                grads = L_model_backward(AL, Y_batch, caches)
                # Backward propagation with L2 reg
                grads = L_model_backward_L2_reg(AL, Y_batch, caches, self.L2_lambd)

                # Update parameters.
                newparams = update_parameters(newparams, grads, self.learning_rate)
                    
            # Print the cost every nth training cycle in epoch
            if self.print_cost and i % 8 == 0:
                endtime = time.time()
                print ("Cost after run %i, iteration %i: %f, Time: %f" %(self.curr_epoch, i, cost, (endtime-self.starttime)))
                self.costs.append(cost)
                self.train_successes.append(train_success)

                # Run xval set and compute cost.
                AL_xval, xval = L_model_forward(X_xval, newparams)        
#                xval_cost, xval_success = compute_cost(AL_xval, Y_xval)
                xval_cost, xval_success = compute_L2_reg_cost(AL_xval, Y_xval, newparams, self.L2_lambd)
                self.xval_successes.append(xval_success)
                
                # plot the costs
                plt.plot(np.squeeze(self.train_successes), 'b')
                plt.plot(np.squeeze(self.xval_successes), 'k')            
                plt.ylabel('Success %')
                plt.xlabel('iterations (per tens)')
                plt.title("Learning rate =" + str(self.learning_rate))
                plt.draw()
                plt.pause(1)
                
                # Optional save parameters every nth iteration for future use
                self.saveParams(newparams)
    
                
        return parameters, newparams, train_success

    def L_layer_model(self, X, Y, newparams):#lr was 0.009
    #    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
        
        # Parameters initialization
        if (len(newparams) == 0):
            newparams = initialize_parameters_deep(self.layers_dims)
                        
        # Batch training, numbatches is 1 if no batch separation
        batch_size = int(X.shape[1]/self.numbatches)

        for k in range(0, self.numbatches):

            X_batch = X[:,(k*batch_size):((k+1)*batch_size-1)]
            Y_batch = Y[(k*batch_size):((k+1)*batch_size-1)]

            # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.            
            AL, caches = L_model_forward(X_batch, newparams)

            # Compute cost
#                cost, train_success = compute_cost(AL, Y_batch)
            # Compute cost with L2 reg
            cost, train_success = compute_L2_reg_cost(AL, Y_batch, newparams, self.L2_lambd)

            # Backward propagation
#                grads = L_model_backward(AL, Y_batch, caches)
            # Backward propagation with L2 reg
            grads = L_model_backward_L2_reg(AL, Y_batch, caches, self.L2_lambd)

            # Update parameters.
            newparams = update_parameters(newparams, grads, self.learning_rate)

            # Print the cost every nth training cycle in epoch
            if self.print_cost and k % 10 == 0:
                endtime = time.time()
                print ("Cost after run %i, iteration %i: %f, Time: %f" %(self.curr_epoch, k, cost, (endtime-self.starttime)))
                self.costs.append(cost)
                self.train_successes.append(train_success)

                # plot the costs
                plt.plot(np.squeeze(self.train_successes), 'b')
                plt.ylabel('Success %')
                plt.xlabel('iterations (per tens)')
                plt.title("Learning rate =" + str(self.learning_rate))
                plt.draw()
                plt.pause(1)

                # Optional save parameters every nth iteration for future use
                self.saveParams(newparams)
    
                
        return newparams, newparams, train_success
          
    def train_data(self):
        
        print('Layer dims...'+str(self.layers_dims) + '...' + str(time.ctime()))

        parameters = {}
        if (self.use_saved_params):
            parameters = self.loadSavedParams()
 
        for k in range(self.epochs):
          
          self.curr_epoch = k
     
          for i in range(self.numtrainfiles):

              if (self.numtrainfiles) > 1:
                  X_filename = str.split(self.train_data_file,'.')[0] + '_' + str(i+1) + '.' + str.split(self.train_data_file,'.')[1]
                  Y_filename = str.split(self.train_labels_file,'.')[0] + '_' + str(i+1) + '.' + str.split(self.train_labels_file,'.')[1]
              else:
                  X_filename = self.train_data_file
                  Y_filename = self.train_labels_file 

              # Load dataset 
              X_train = np.loadtxt(self.localfolder + X_filename)
              Y_train = np.loadtxt(self.localfolder + Y_filename)

              # Number of examples
              m = X_train.shape[1]
              print('Data loaded..' + str(time.ctime()) + '...' + str(X_train.shape))

              # Shuffle the data
              marr = np.arange(m)
              np.random.shuffle(marr)
              X_train = X_train[:,marr]
              Y_train = Y_train[marr]

              X_train = X_train/255       
              
              # After the first run, start using the derived params
              if k > 0 or i > 0:
                  parameters = newparams

              params, newparams, train_success = self.L_layer_model(X_train, Y_train, parameters)

        print('Optimization done..'+str(train_success))        
        self.saveParams(newparams)
    
            
    def test_data(self, t_size):
        
        # Load test data
        X_test = np.loadtxt(self.localfolder + self.test_data_file)
        Y_test = np.loadtxt(self.localfolder + self.test_labels_file)
        
        # Number of examples
        m = X_test.shape[1]
        
        # Shuffle the data
        marr = np.arange(m)
        np.random.shuffle(marr)
        X_test = X_test[:,marr]
        Y_test = Y_test[marr]
        
        X_test = X_test/255
        
        parameters = self.loadSavedParams()
        
        ### Run forward prop to get the output activations    
        AL, caches = L_model_forward(X_test, parameters)
        
        ### Convert AL to binary calls
        calls = (AL >= 0.5)
        
        ## Calculate success percentage
        success = 1 - np.count_nonzero(Y_test - calls)/m
    
        # Show images from test data and the classification results    
#        self.showImageData(X_test, calls, (t_size))    
        
        return success


In [12]:
myNN = Deep_NN_frmwrk("catsDogs")
print(myNN.kfolds)
print(myNN.localfolder)

starttime = time.time()
print(time.ctime())

myNN.train_data()

### Test data
success = myNN.test_data((myNN.img_w, myNN.img_h))

endtime = time.time()
print('Time {0}, num_epochs {1}, success {2}'.format((endtime-starttime), myNN.num_epochs, success))


20
gdrive/My Drive/
Mon Apr 15 01:57:51 2019
Layer dims...[16384, 25, 13, 5, 1]...Mon Apr 15 01:57:51 2019
Data loaded..Mon Apr 15 02:00:00 2019...(16384, 8012)
Cost after run 0, iteration 0: 0.760293, Time: 133.601445


NameError: ignored